In [1]:
from sparksampling import Submitter
submitter = Submitter()

In [2]:
from sparksampling.utilities.var import FILE_TYPE_CSV
from sparksampling.utilities.var import SIMPLE_RANDOM_SAMPLING_METHOD

submit_response = submitter.submit_sampling_simplejob('hdfs://localhost:9000/dataset/titanic/train.csv',
                                              method=SIMPLE_RANDOM_SAMPLING_METHOD,
                                              file_type=FILE_TYPE_CSV,
                                              with_header=True)
job_id = submit_response.data['job_id']
submit_response

request: http://localhost:8000/v1/sampling/simplejob/
2021-01-26 10:31:11,366 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-01-26 10:31:11,419 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/sampling/simplejob/ HTTP/1.1" 200 49


In [3]:
submit_response.to_dict()

{'code': 0, 'msg': '', 'data': {'job_id': 10000}}

In [14]:
submitter.get_sampling_job_details(job_id)

request: http://localhost:8000/v1/sampling/query/job/


{'code': 0,
 'msg': '',
 'data': {'job_id': 10007,
  'job_status': 'Succeed',
  'msg': 'succeed',
  'method': 'Simple Random Sampling',
  'start_time': '2021/01/22/ 10:13:49',
  'end_time': '2021/01/22 10:13:50',
  'simpled_file_path': 'hdfs://localhost:9000/dataset/titanic/train.csv-sampled-1611281629.4054315',
  'request_data': "{'path': 'hdfs://localhost:9000/dataset/titanic/train.csv', 'method': 1, 'type': 2, 'with_header': True, 'conf': {}}"}}

In [15]:
data = submitter.get_statistics(job_id=job_id, from_sampling=True, file_type=FILE_TYPE_CSV,with_header=True)['data']
data

request: http://localhost:8000/v1/evaluation/statistics/


[{'summary': 'count',
  'PassengerId': '478',
  'Survived': '478',
  'Pclass': '478',
  'Name': '478',
  'Sex': '478',
  'Age': '391',
  'SibSp': '478',
  'Parch': '478',
  'Ticket': '478',
  'Fare': '478',
  'Cabin': '110',
  'Embarked': '478'},
 {'summary': 'mean',
  'PassengerId': '439.9979079497908',
  'Survived': '0.4100418410041841',
  'Pclass': '2.288702928870293',
  'Name': None,
  'Sex': None,
  'Age': '30.155601023017905',
  'SibSp': '0.5334728033472803',
  'Parch': '0.4246861924686193',
  'Ticket': '301151.71142857143',
  'Fare': '34.130464644351434',
  'Cabin': None,
  'Embarked': None},
 {'summary': 'stddev',
  'PassengerId': '260.25032699310214',
  'Survived': '0.49235624632787073',
  'Pclass': '0.8444652654311523',
  'Name': None,
  'Sex': None,
  'Age': '14.128870337349701',
  'SibSp': '1.1482960077789233',
  'Parch': '0.9189236969722622',
  'Ticket': '547604.6362277074',
  'Fare': '56.180566079429475',
  'Cabin': None,
  'Embarked': None},
 {'summary': 'min',
  'Passen

In [16]:
import pandas as pd
pd.DataFrame.from_records(data, index='summary')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,478,478,478,478,478,391,478,478,478,478,110,478
mean,439.9979079497908,0.4100418410041841,2.288702928870293,None,None,30.155601023017905,0.5334728033472803,0.4246861924686193,301151.71142857143,34.130464644351434,None,None
stddev,260.25032699310214,0.49235624632787073,0.8444652654311523,None,None,14.128870337349701,1.1482960077789233,0.9189236969722622,547604.6362277074,56.180566079429475,None,None
min,1,0,1,"""Bradley, Mr. George (""""George Arthur Brayton"""")""",female,0.67,0,0,110152,0,A14,C
max,99,1,3,"van Billiard, Mr. Austin Blyler",male,9,8,5,WE/P 5735,93.5,T,S
